In [1]:
# Check what variables exist in your notebook
print("Available variables:")
for var_name in dir():
    if not var_name.startswith('_'):
        print(f"- {var_name}")

# Check specifically for documents-related variables
if 'documents' in locals():
    print(f"\n✓ documents variable exists with {len(documents)} items")
else:
    print("\n✗ documents variable not found")
    
if 'success_list' in locals():
    print(f"✓ success_list exists with {len(success_list)} files")
    
if 'failure_list' in locals():
    print(f"✓ failure_list exists with {len(failure_list)} files")

Available variables:
- In
- Out
- exit
- get_ipython
- open
- quit

✗ documents variable not found


In [ ]:
# Import required libraries
#from the documents loading all pdf files from the data/documents folder
import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

def load_pdf_files():
    """
    Load all PDF files from data/documents folder
    """
    base_folder = Path('../data/documents')
    all_documents = []
    
    # Find all PDF files
    pdf_files = list(base_folder.rglob("*.pdf"))
    print(f"Found {len(pdf_files)} PDF files")
    
    for i, pdf_file in enumerate(pdf_files, 1):
        print(f"[{i}/{len(pdf_files)}] Loading: {pdf_file.name}")
        
        try:
            loader = PyPDFLoader(str(pdf_file))
            docs = loader.load()
            
            # Add source file metadata
            for doc in docs:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_path'] = str(pdf_file)
            
            all_documents.extend(docs)
            print(f"  ✓ Loaded {len(docs)} pages")
            
        except Exception as e:
            print(f"  ✗ Failed: {e}")
    
    print(f"\nTotal: {len(all_documents)} pages loaded")
    return all_documents

# Load documents
print("Loading PDF documents...")
documents = load_pdf_files()

# Verify loading
if documents:
    print(f"\n✅ SUCCESS: {len(documents)} pages ready for chunking")
    
    # Show files loaded
    files_loaded = set(doc.metadata.get('source_file') for doc in documents)
    print(f"Files loaded: {len(files_loaded)}")
    for file in files_loaded:
        print(f"  - {file}")
else:
    print("❌ No documents loaded")

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 78 0 (offset 0)


Loading PDF documents...
Found 7 PDF files
[1/7] Loading: SJ_Developers.pdf


Ignoring wrong pointing object 97 0 (offset 0)
Ignoring wrong pointing object 99 0 (offset 0)


  ✓ Loaded 9 pages
[2/7] Loading: Sri Sathya Sai Serenity_Layout_Final.pdf
  ✓ Loaded 1 pages
[3/7] Loading: AmariFarms_Brochure.pdf
  ✓ Loaded 30 pages
[4/7] Loading: VILLA 01.pdf
  ✓ Loaded 2 pages
[5/7] Loading: VILLA 02.pdf
  ✓ Loaded 2 pages
[6/7] Loading: REGISTRATION OF FIRMS.pdf
  ✓ Loaded 1 pages
[7/7] Loading: CIBW1022010.pdf
  ✓ Loaded 17 pages

Total: 62 pages loaded

✅ SUCCESS: 62 pages ready for chunking
Files loaded: 7
  - SJ_Developers.pdf
  - VILLA 01.pdf
  - CIBW1022010.pdf
  - AmariFarms_Brochure.pdf
  - VILLA 02.pdf
  - Sri Sathya Sai Serenity_Layout_Final.pdf
  - REGISTRATION OF FIRMS.pdf


In [ ]:
# Use your already loaded documents to create chunks
#making chunks from the loaded documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
  # Assuming you have a data_loader module to load your documents

# Configure chunk settings
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

def create_chunks_from_documents(documents):
    """
    Create chunks from your loaded PDF documents
    """
    print(f"Creating chunks from {len(documents)} loaded pages...")
    
    # Initialize text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    
    # Split documents into chunks
    chunks = text_splitter.split_documents(documents)
    
    print(f"✓ Successfully created {len(chunks)} chunks")
    
    # Show chunks distribution by source file
    chunk_stats = {}
    for chunk in chunks:
        source = chunk.metadata.get('source_file', 'Unknown')
        chunk_stats[source] = chunk_stats.get(source, 0) + 1
    
    print("\n📊 CHUNKS BY FILE:")
    print("-" * 40)
    for file, count in chunk_stats.items():
        print(f"  {file}: {count} chunks")
    
    return chunks

# Create chunks from your loaded documents
chunks = create_chunks_from_documents(documents)

# Show sample chunk
if chunks:
    print(f"\n📄 SAMPLE CHUNK:")
    print("-" * 50)
    print(f"File: {chunks[0].metadata.get('source_file')}")
    print(f"Page: {chunks[0].metadata.get('page', 'N/A')}")
    print(f"Length: {len(chunks[0].page_content)} characters")
    print(f"Content preview:\n{chunks[0].page_content[:300]}...")

Creating chunks from 62 loaded pages...
✓ Successfully created 83 chunks

📊 CHUNKS BY FILE:
----------------------------------------
  SJ_Developers.pdf: 10 chunks
  AmariFarms_Brochure.pdf: 8 chunks
  VILLA 01.pdf: 2 chunks
  VILLA 02.pdf: 2 chunks
  CIBW1022010.pdf: 61 chunks

📄 SAMPLE CHUNK:
--------------------------------------------------
File: SJ_Developers.pdf
Page: 0
Length: 48 characters
Content preview:
SJ DevelopersAnantapuram, Andhra Pradesh., India...


In [4]:
#storing chunks in ChromaDB vector database
# Install required packages
# Ensure you have the necessary packages installed
#this one not worked out used second approach
%pip install sentence-transformers

import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os

def store_in_chromadb(chunks):
    """
    Store chunks in ChromaDB vector database
    """
    print("Setting up ChromaDB vector store...")
    
    # Create embeddings model
    print("Loading embedding model...")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    
    # Create vector store directory
    vector_store_path = '../vector_store'
    os.makedirs(vector_store_path, exist_ok=True)
    
    # Create ChromaDB vector store
    print("Creating ChromaDB vector store...")
    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=vector_store_path,
        collection_name="sj_developers_knowledge"
    )
    
    # Persist to disk
    vectordb.persist()
    
    print(f"✅ ChromaDB created successfully!")
    print(f"📁 Location: {vector_store_path}")
    print(f"📊 Total vectors: {len(chunks)}")
    print(f"🏷️  Collection: sj_developers_knowledge")
    
    return vectordb

# Store chunks in ChromaDB
if 'chunks' in locals() and chunks:
    vectordb = store_in_chromadb(chunks)
else:
    print("❌ No chunks available for storage")

Note: you may need to restart the kernel to use updated packages.
Setting up ChromaDB vector store...
Loading embedding model...


/var/folders/x5/gxfv5zh57dd8qw9f49g4t1rh0000gn/T/ipykernel_9826/532510386.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/anaconda3/env

Creating ChromaDB vector store...


RuntimeError: Numpy is not available

In [5]:
# Simple approach using basic ChromaDB without external embeddings
#Let's use a simpler approach without sentence-transformers that might have compatibility issues:


import chromadb
import os
from pathlib import Path

def create_simple_chromadb(chunks):
    """
    Create ChromaDB using default embeddings (no external dependencies)
    """
    print("Creating simple ChromaDB vector store...")
    
    # Create vector store directory
    vector_store_path = Path('../vector_store')
    vector_store_path.mkdir(exist_ok=True, parents=True)
    
    # Initialize ChromaDB client
    client = chromadb.PersistentClient(path=str(vector_store_path))
    
    # Create or get collection
    collection_name = "sj_developers_knowledge"
    
    try:
        # Try to get existing collection
        collection = client.get_collection(collection_name)
        print(f"Found existing collection: {collection_name}")
    except:
        # Create new collection
        collection = client.create_collection(collection_name)
        print(f"Created new collection: {collection_name}")
    
    # Prepare documents for ChromaDB
    documents = []
    metadatas = []
    ids = []
    
    for i, chunk in enumerate(chunks):
        documents.append(chunk.page_content)
        metadatas.append({
            'source_file': chunk.metadata.get('source_file', 'unknown'),
            'page': str(chunk.metadata.get('page', 0)),
            'chunk_id': i
        })
        ids.append(f"chunk_{i}")
    
    # Add documents to collection
    print(f"Adding {len(documents)} documents to ChromaDB...")
    
    # Add in batches to avoid memory issues
    batch_size = 100
    for i in range(0, len(documents), batch_size):
        batch_docs = documents[i:i+batch_size]
        batch_meta = metadatas[i:i+batch_size]
        batch_ids = ids[i:i+batch_size]
        
        collection.add(
            documents=batch_docs,
            metadatas=batch_meta,
            ids=batch_ids
        )
        print(f"Added batch {i//batch_size + 1}/{(len(documents)-1)//batch_size + 1}")
    
    print(f"✅ ChromaDB created successfully!")
    print(f"📁 Location: {vector_store_path}")
    print(f"📊 Total documents: {collection.count()}")
    
    return client, collection

# Create ChromaDB
if 'chunks' in locals() and chunks:
    client, collection = create_simple_chromadb(chunks)
    
    # Test the database
    print("\n🔍 Testing search functionality...")
    results = collection.query(
        query_texts=["What is SJ Developers?"],
        n_results=3
    )
    
    print(f"Found {len(results['documents'][0])} results:")
    for i, doc in enumerate(results['documents'][0]):
        print(f"\nResult {i+1}:")
        print(f"Source: {results['metadatas'][0][i]['source_file']}")
        print(f"Content: {doc[:150]}...")
        
else:
    print("❌ No chunks available. Run chunking code first.")

Creating simple ChromaDB vector store...
Found existing collection: sj_developers_knowledge
Adding 83 documents to ChromaDB...


/Users/SreeChow/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:35<00:00, 2.32MiB/s]


Added batch 1/1
✅ ChromaDB created successfully!
📁 Location: ../vector_store
📊 Total documents: 83

🔍 Testing search functionality...
Found 3 results:

Result 1:
Source: SJ_Developers.pdf
Content: SJ DevelopersAnantapuram, Andhra Pradesh., India...

Result 2:
Source: SJ_Developers.pdf
Content: SJ Developers is a real-estate development company, which is registered at Anantapuram, Andhra Pradesh, India.What we do?vFirst, we make a development...

Result 3:
Source: CIBW1022010.pdf
Content: trial project is a commercial building in “open book”, meaning the client is another company from the 
same Group Corporation. The system was presente...


In [6]:
#this is a function to search the knowledge base
def search_knowledge_base(client, collection_name, query, n_results=3):
    """
    Search the knowledge base
    """
    collection = client.get_collection(collection_name)
    
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    
    return results

# Test different queries
if 'client' in locals() and 'collection' in locals():
    test_queries = [
        "What is SJ Developers?",
        "Tell me about villas",
        "Registration information",
        "Amari Farms details"
    ]
    
    print("\n🔍 TESTING KNOWLEDGE BASE:")
    print("=" * 50)
    
    for query in test_queries:
        print(f"\nQuery: '{query}'")
        print("-" * 30)
        
        results = search_knowledge_base(client, "sj_developers_knowledge", query, 2)
        
        for i, doc in enumerate(results['documents'][0]):
            source = results['metadatas'][0][i]['source_file']
            print(f"\nResult {i+1}: {source}")
            print(f"Content: {doc[:200]}...")


🔍 TESTING KNOWLEDGE BASE:

Query: 'What is SJ Developers?'
------------------------------

Result 1: SJ_Developers.pdf
Content: SJ DevelopersAnantapuram, Andhra Pradesh., India...

Result 2: SJ_Developers.pdf
Content: SJ Developers is a real-estate development company, which is registered at Anantapuram, Andhra Pradesh, India.What we do?vFirst, we make a development agreement with the owner of the open land and tak...

Query: 'Tell me about villas'
------------------------------

Result 1: VILLA 02.pdf
Content: LIVING14'11"X13'6"
BED ROOM10'1"X10'4"M BED ROOM11'9"X10'4"
G BATH4'6"X7'0"
A BATH4'0"X8'0"
PARKING12'5"X12'4"
3'
2'
2'
D D
ED
D1
D1 2'
CLIENT:SOLLAPURAM,ANANTAPURDIST.
LOCATION:
ENGINEERS - DESIGNERS...

Result 2: VILLA 01.pdf
Content: UPLIVING14'8"X12'0"
KITCHEN12'3"X6'7"
DINING11'11"x10'7"
BED ROOM9'11"X9'9"M BED ROOM11'11"X9'9"
G BATH4'0"X6'5"
A BATH4'0"X6'0"
PARKING13'3"x13'5"3'6" WIDEUTILITY
POOJA3'6"X4'0"
3'
2'-6"
2'-6"
T V UN...

Query: 'Registration information'
------

In [7]:
import gradio as gr
import chromadb
from pathlib import Path

def load_knowledge_base():
    """
    Load the existing ChromaDB knowledge base
    """
    try:
        vector_store_path = Path('../vector_store')
        client = chromadb.PersistentClient(path=str(vector_store_path))
        collection = client.get_collection("sj_developers_knowledge")
        print(f"✅ Knowledge base loaded: {collection.count()} documents")
        return client, collection
    except Exception as e:
        print(f"❌ Error loading knowledge base: {e}")
        return None, None

def search_documents(query, num_results=3):
    """
    Search the knowledge base and return formatted results
    """
    if not query.strip():
        return "Please enter a question to search the knowledge base."
    
    try:
        # Load knowledge base
        client, collection = load_knowledge_base()
        
        if not collection:
            return "❌ Knowledge base not found. Please create the vector database first."
        
        # Search the collection
        results = collection.query(
            query_texts=[query],
            n_results=num_results
        )
        
        if not results['documents'][0]:
            return "No relevant documents found for your query."
        
        # Format results
        response = f"🔍 **Search Results for:** *{query}*\n\n"
        response += f"📊 **Found {len(results['documents'][0])} relevant documents:**\n\n"
        
        for i, doc in enumerate(results['documents'][0]):
            source = results['metadatas'][0][i]['source_file']
            page = results['metadatas'][0][i].get('page', 'N/A')
            
            response += f"### 📄 Result {i+1}: {source}\n"
            response += f"**Page:** {page}\n\n"
            response += f"**Content:**\n{doc[:400]}...\n\n"
            response += "---\n\n"
        
        return response
        
    except Exception as e:
        return f"❌ Error searching knowledge base: {str(e)}"

def get_knowledge_stats():
    """
    Get statistics about the knowledge base
    """
    try:
        client, collection = load_knowledge_base()
        if collection:
            count = collection.count()
            return f"📊 Knowledge Base contains {count} document chunks"
        else:
            return "❌ Knowledge base not available"
    except:
        return "❌ Error accessing knowledge base"

# Create Gradio Interface
def create_gradio_interface():
    """
    Create the Gradio web interface
    """
    with gr.Blocks(title="SJ Developers Knowledge Worker", theme=gr.themes.Soft()) as app:
        
        # Header
        gr.Markdown("""
        # 🏢 SJ Developers Knowledge Worker
        ### Ask questions about your PDF documents
        """)
        
        # Knowledge base stats
        stats = get_knowledge_stats()
        gr.Markdown(f"**Status:** {stats}")
        
        # Main interface
        with gr.Row():
            with gr.Column(scale=2):
                # Input section
                query_input = gr.Textbox(
                    label="🔍 Ask a question",
                    placeholder="e.g., What is SJ Developers? Tell me about villas...",
                    lines=2
                )
                
                num_results = gr.Slider(
                    minimum=1,
                    maximum=5,
                    value=3,
                    step=1,
                    label="Number of results"
                )
                
                search_btn = gr.Button("🔍 Search Knowledge Base", variant="primary")
                clear_btn = gr.Button("🧹 Clear", variant="secondary")
        
        # Output section
        with gr.Row():
            output = gr.Markdown(
                label="📋 Search Results",
                value="Enter a question above to search the knowledge base."
            )
        
        # Example queries
        gr.Markdown("""
        ### 💡 Example Questions:
        - What is SJ Developers?
        - Tell me about the available villas
        - What are the registration details?
        - Information about Amari Farms
        - What properties are available?
        """)
        
        # Event handlers
        search_btn.click(
            fn=search_documents,
            inputs=[query_input, num_results],
            outputs=output
        )
        
        clear_btn.click(
            fn=lambda: ("", "Enter a question above to search the knowledge base."),
            outputs=[query_input, output]
        )
        
        # Allow Enter key to trigger search
        query_input.submit(
            fn=search_documents,
            inputs=[query_input, num_results],
            outputs=output
        )
    
    return app

# Launch the interface
print("Creating Gradio interface...")
app = create_gradio_interface()

# Launch with public sharing (optional)
print("Launching knowledge worker interface...")
app.launch(
    share=False,  # Set to True if you want a public link
    server_name="127.0.0.1",
    server_port=7860,
    show_error=True
)

Creating Gradio interface...
✅ Knowledge base loaded: 83 documents
Launching knowledge worker interface...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


✅ Knowledge base loaded: 83 documents


In [8]:
# Enhanced version with chat-like interface
import gradio as gr
import chromadb
from pathlib import Path
from datetime import datetime

def create_chat_interface():
    """
    Create a chat-like interface for the knowledge base
    """
    def chat_with_knowledge_base(message, history):
        """
        Process chat message and return response with history
        """
        if not message.strip():
            return history, ""
        
        try:
            # Load knowledge base
            vector_store_path = Path('../vector_store')
            client = chromadb.PersistentClient(path=str(vector_store_path))
            collection = client.get_collection("sj_developers_knowledge")
            
            # Search for relevant documents
            results = collection.query(
                query_texts=[message],
                n_results=2
            )
            
            if results['documents'][0]:
                # Format response
                response = "Based on your documents:\n\n"
                
                for i, doc in enumerate(results['documents'][0]):
                    source = results['metadatas'][0][i]['source_file']
                    response += f"📄 **From {source}:**\n"
                    response += f"{doc[:300]}...\n\n"
                
                # Add to history
                history.append([message, response])
            else:
                response = "I couldn't find relevant information in your documents for that question."
                history.append([message, response])
                
        except Exception as e:
            response = f"Error searching knowledge base: {str(e)}"
            history.append([message, response])
        
        return history, ""
    
    # Create chat interface
    with gr.Blocks(title="SJ Developers Chat", theme=gr.themes.Soft()) as chat_app:
        gr.Markdown("# 💬 Chat with SJ Developers Knowledge Base")
        
        chatbot = gr.Chatbot(
            label="Knowledge Worker Assistant",
            height=400,
            show_label=True
        )
        
        with gr.Row():
            msg = gr.Textbox(
                label="Message",
                placeholder="Ask me anything about your documents...",
                scale=4
            )
            submit = gr.Button("Send", variant="primary", scale=1)
        
        clear = gr.Button("Clear Chat")
        
        # Event handlers
        submit.click(
            chat_with_knowledge_base,
            inputs=[msg, chatbot],
            outputs=[chatbot, msg]
        )
        
        msg.submit(
            chat_with_knowledge_base,
            inputs=[msg, chatbot],
            outputs=[chatbot, msg]
        )
        
        clear.click(lambda: [], outputs=chatbot)
    
    return chat_app

# Launch chat interface
print("Creating chat interface...")
chat_app = create_chat_interface()
chat_app.launch(share=False, server_port=7861)

Creating chat interface...


/var/folders/x5/gxfv5zh57dd8qw9f49g4t1rh0000gn/T/ipykernel_9826/3623651953.py:55: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
